In [5]:
from langchain.output_parsers import YamlOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import AzureChatOpenAI

In [6]:
azure_config = {
# enter credentials here
    }

llm = AzureChatOpenAI(temperature=0,
                      api_key=azure_config["api-key"],
                      openai_api_version=azure_config["api_version"],
                      azure_endpoint=azure_config["base_url"],
                      model=azure_config["model_deployment"],
                      validate_base_url=False)

model = llm

In [9]:
class Poem(BaseModel):
    setup: str = Field(description="question to recite a poem")
    poemline: str = Field(description="answer to recite a poem")

In [10]:
poem_query = "Recite me a poem."

# Set up a parser + inject instructions into the prompt template.
yaml_parser = YamlOutputParser(pydantic_object=Poem)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": yaml_parser.get_format_instructions()},
)

chain = prompt | model | yaml_parser

chain.invoke({"query": poem_query})

Poem(setup='What is the meaning of life?', poemline='Life is a journey, a story to be told, filled with moments of love, and memories to hold.')